In [1]:
import pandas as pd
import numpy as np
import os
import feets
import warnings

## Application test with 1 light curve

In [2]:
df = pd.read_csv('../LightCurves/001152358_22.dat', sep=' ', header=None, names=['time', 'flux'])
df.head()

,time,flux
0,1900.414603,0.955040
1,1900.415992,1.013595
2,1900.417381,0.996042
3,1900.418770,1.031510
4,1900.420158,0.972890


In [4]:
time, flux = (df.time, df.flux)
lc_feets = [time, flux]

In [5]:
%%time
fs = feets.FeatureSpace(data=['magnitude', 'time'], only=['Amplitude', 'Eta_e', 'LinearTrend', 'Mean', 'Meanvariance', 'MedianAbsDev', 'PercentAmplitude', 'PeriodLS', 'Period_fit', 'Psi_CS', 'Psi_eta', 'Std'])
features, values = fs.extract(*lc_feets)
table = pd.DataFrame(data=[values], columns=[features])
table

Wall time: 9.31 s


,Amplitude,Eta_e,LinearTrend,Mean,Meanvariance,MedianAbsDev,PercentAmplitude,PeriodLS,Period_fit,Psi_CS,Psi_eta,Std
0,0.038367,2.781633,0.000008,0.999896,0.01949,0.013162,0.067496,0.000091,1.0,0.013116,2.01383,0.019488


## Application with a test data set

In [14]:
input_dir = os.listdir('LC_test')

dataframes = []

for file in input_dir:
    id_sec = file.split('.')[0] 
    id_name, sec_name = id_sec.split('_')

    df_1 = pd.DataFrame([[id_name, sec_name]], columns=['id', 'sector'])
    dataframes.append(df_1)

df_final = pd.concat(dataframes, ignore_index=True)

df_final

,id,sector
0,001152358,22
1,004289780,06
2,004289780,32
3,004373827,20
4,004373827,44
5,004373827,45
6,004373827,46
7,004373827,47
8,004373827,60
9,004491131,20


In [ ]:
warnings.simplefilter("ignore")

# Charge the files
df_class = pd.read_csv('TIC_Classes_List_v2.csv', sep=';')
input_dir = os.listdir('LC_test')

# Create a longtable with the features from feets
table = []

for file in input_dir:
    df = pd.read_csv(f'LC_test/{file}', sep=' ', header=None, names=['time', 'flux'])
    time, flux = (df.time, df.flux)
    lc_feets = [time, flux]

    fs = feets.FeatureSpace(data=['magnitude', 'time'], only=['Amplitude', 'Eta_e', 'LinearTrend', 'Mean', 'Meanvariance', 'MedianAbsDev', 'PercentAmplitude', 'PeriodLS', 'Period_fit', 'Psi_CS', 'Psi_eta', 'Std'])
    features, values = fs.extract(*lc_feets)

    id_sec = file.split('.')[0] 
    id_name, sec_name = id_sec.split('_')

    df_1 = pd.DataFrame(data=[[id_name, sec_name, *values]], columns=['id', 'sector']+list(features))
    table.append(df_1)

df_main = pd.concat(table, ignore_index=True)

# Create a second DF with the classes from df_class file and merge to df_main
old_column_name = df_class.columns[0]
df_class.rename(columns={old_column_name: 'id'}, inplace=True)

df_main['id'] = df_main['id'].astype(str)
df_class['id'] = df_class['id'].astype(str)

feets_final_table = df_main.merge(df_class, on='id', how='left')
print(feets_final_table)

feets_final_table.to_csv('feets_full_table_exemple.csv', sep=';', index=False)

In [ ]:
def feets_table_generation(input_folder, input_file):
    warnings.simplefilter("ignore")

    df_class = pd.read_csv(input_file, sep=';')
    input_dir = os.listdir(input_folder)

    table = []

    for file in input_dir:
        df = pd.read_csv(f'LC_test/{file}', sep=' ', header=None, names=['time', 'flux'])
        time, flux = (df.time, df.flux)
        lc_feets = [time, flux]

        fs = feets.FeatureSpace(data=['magnitude', 'time'], only=['Amplitude', 'Eta_e', 'LinearTrend', 'Mean', 'Meanvariance', 'MedianAbsDev', 'PercentAmplitude', 'PeriodLS', 'Period_fit', 'Psi_CS', 'Psi_eta', 'Std'])
        features, values = fs.extract(*lc_feets)

        id_sec = file.split('.')[0] 
        id_name, sec_name = id_sec.split('_')

        df_1 = pd.DataFrame(data=[[id_name, sec_name, *values]], columns=['id', 'sector']+list(features))
        table.append(df_1)

    df_main = pd.concat(table, ignore_index=True)

    # Create a second DF with the classes from df_class file and merge to df_main
    old_column_name = df_class.columns[0]
    df_class.rename(columns={old_column_name: 'id'}, inplace=True)

    df_main['id'] = df_main['id'].astype(str)
    df_class['id'] = df_class['id'].astype(str)

    feets_final_table = df_main.merge(df_class, on='id', how='left')

    feets_final_table.to_csv('feets_full_table_exemple.csv', sep=';', index=False)


In [3]:
input_folder = input('Insert folder full adress: ')
input_file = input('Insert file full adress: ')

feets_table_generation(input_folder, input_file)